In [1]:
from datasets import load_dataset
import datasets
from transformers import AutoTokenizer, DataCollatorWithPadding
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

2022-04-20 13:17:12.997499: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/sw/spack/apps/linux-centos8-cascadelake/gcc-9.3.0/julia-1.5.2-3z/lib:/opt/ohpc/pub/libs/hwloc/2.1.0/lib:/opt/ohpc/pub/mpi/libfabric/1.10.1/lib:/opt/ohpc/pub/mpi/ucx-ohpc/1.8.0/lib:/opt/ohpc/pub/mpi/openmpi4-gnu9/4.0.4/lib:/opt/ohpc/pub/compiler/gcc/9.3.0/lib64
2022-04-20 13:17:12.997523: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dataset = load_dataset('csv', data_files={'train': ['train.csv'], 'test': 'test.csv', 'validation':'Validate.csv'})

Using custom data configuration default-cecefb37d251f8fd


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/dnarendr/.cache/huggingface/datasets/csv/default-cecefb37d251f8fd/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
def tokenize_function(example):
    try:
      return tokenizer(example["text"], truncation=True)
    except  Exception as e:
      print(e)

In [4]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
print("tokenized_datasets::", tokenized_datasets)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

tokenized_datasets:: DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5473
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1524
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 610
    })
})


In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="sentModel", evaluation_strategy="steps", eval_steps=500, num_train_epochs=3)

In [6]:
import numpy as np

In [7]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


def compute_metrics(eval_pred):    
    pred, labels = eval_pred
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=4)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [12]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/dnarendr/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5473
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2055


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,1.134300,1.163440,0.511475,0.496485,0.511475,0.488995
1000,0.665900,1.352382,0.555738,0.566185,0.555738,0.553604
1500,0.418800,1.477529,0.649180,0.653978,0.649180,0.650775
2000,0.269900,1.552945,0.637705,0.654153,0.637705,0.640957


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 610
  Batch size = 8
Saving model checkpoint to sentModel/checkpoint-500
Configuration saved in sentModel/checkpoint-500/config.json
Model weights saved in sentModel/checkpoint-500/pytorch_model.bin
tokenizer config file saved in sentModel/checkpoint-500/tokenizer_config.json
Special tokens file saved in sentModel/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 610
  Batch size = 8
Saving mod

TrainOutput(global_step=2055, training_loss=0.6135490788740543, metrics={'train_runtime': 1798.892, 'train_samples_per_second': 9.127, 'train_steps_per_second': 1.142, 'total_flos': 476952852408864.0, 'train_loss': 0.6135490788740543, 'epoch': 3.0})

In [13]:
model.save_pretrained("my_model")

Configuration saved in my_model/config.json
Model weights saved in my_model/pytorch_model.bin


In [14]:
from transformers import pipeline
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('my_model', local_files_only=True)
classifier = pipeline(task = 'text-classification', model = model, tokenizer=tokenizer)


loading configuration file my_model/config.json
Model config BertConfig {
  "_name_or_path": "my_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights f

In [15]:
classifier(["Your Zions First National Bank Card has been issued by Zions Bancorporation, NA dba Zions First National Bank.", "Your Account is with Zions Bancorporation, NA (Bank) and will be administered by the Banks Bankcard Services department.", "Your Credit Card, monthly statement, and other associated materials will bear the Zions First National Bank, National Bank of Arizona, Nevada State Bank, or Vectra Bank Colorado name.", "The Card is the property of the Bank."])

Disabling tokenizer parallelism, we're using DataLoader multithreading already


[{'label': 'LABEL_1', 'score': 0.9908129572868347},
 {'label': 'LABEL_1', 'score': 0.9354974031448364},
 {'label': 'LABEL_0', 'score': 0.7992977499961853},
 {'label': 'LABEL_1', 'score': 0.8896874189376831}]

In [16]:
model_predictions = trainer.predict(tokenized_datasets['test'])
import numpy as np

preds = np.argmax(model_predictions.predictions, axis=-1)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1679
  Batch size = 8


In [25]:
final_score = metric.compute(model_predictions=preds, references='model_predictions.label')

TypeError: 'NoneType' object is not iterable